In [1]:
import pandas as pd
import numpy as np
import os
import requests
from io import BytesIO
import json
import yadisk
from datetime import datetime, date, timedelta
import re
import gspread
from oauth2client.service_account import ServiceAccountCredentials

import config
from yandex_disk_func import *
from parse_functions import *
from google_connector import *

# # указываем путь и название файла с токеном для Яндекс Диск
file_path = r'C:\Users\o.bogomolov\Desktop\Jupyter_notebook\35_mediaplan_parser'
# забираем токен для подключения к гугл
service_key = config.service
google_sheet_link = 'https://docs.google.com/spreadsheets/d/11hF_txYEbKrQImRQ0x3Fia3PyJvDAqtpGI25idV7oa4/edit?usp=sharing'
gmail = config.gmail
sheet_name = 'prog'

# забираем Яндекс токен
yandex_token = config.yandex_token
# # указываем путь к основной папке, в которой храняться папки с флайтами
main_folder = config.main_folder
public_key = config.public_key # обычная ссылка на доступ к папке одного данного ФЛАЙТА из личного кабинета

# забираем список папок в нужной нам директории
res = get_yandex_disk_folders(public_key)
yandex_folders = res.json() # парсим ответ

200


In [2]:
# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None
# pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
# создаем словарь, где сохраним каждый продукт в отдельный датаФрейм со своим ключом
main_dict = {}

In [4]:
get_data_from_ya_folder(main_folder, yandex_folders, yandex_token, main_dict, flag='prog')                     


/01_yandex
/02_prog
mp_mobidriven_игроник_ингосстрах_ хореографы_24.09 (1)
мп_4 дня
мп_4 неделе
mp_mobidriven_игроник_ингосстрах_инго экосистема_05.09
мп
mp_mobidriven_игроник_национальная лотерия_27.08 (1)
мп
/03_Полина


In [5]:
df = pd.concat(main_dict, ignore_index=True)
df = df.fillna('')


int_columns = ['reach', 'impressions', 'clicks']
df[int_columns] = df[int_columns].apply(lambda x: x.astype('int'))

In [6]:
float_columns = ['unit price', 'budget_without_nds', 'budget_nds', 'vtr, %']
df[float_columns] = df[float_columns].apply(lambda x: x.astype('float')).round(2)

In [19]:
test = df[df['report_name']=='mp_mobidriven_игроник_национальная лотерия_27.08 (1)']

In [18]:
test = df[df['sheet_name']=='мп']

In [20]:
test

10,supplier,report_name,sheet_name,brand,period,source,site/ssp,placement,targeting,geo,soc_dem,ad copy format,rotation type,unit price,frequency,reach,impressions,clicks,budget_without_nds,budget_nds,views,"vtr, %"
4,mobidriven,mp_mobidriven_игроник_национальная лотерия_27.08 (1),мп,Национальная Лотерия,01.11.24-30.04.25,Mobidriven,,in app,Ж 35-45 BC. Семейные с детьми. Работают в офисах или на гос. службе. Интересуются скидками и акциями.,РФ,Ж 35-45 лет,Fullscreen,CPM,290.0,3,6000000,18000000,270000,5220000.0,6264000.0,0,0.0
5,mobidriven,mp_mobidriven_игроник_национальная лотерия_27.08 (1),мп,Национальная Лотерия,01.11.24-30.04.25,Mobidriven,,in web,Ж 35-45 BC. Семейные с детьми. Работают в офисах или на гос. службе. Интересуются скидками и акциями.,РФ,Ж 35-45 лет,"Banners (300*250, 320*100, 320*50)",CPM,110.0,3,5000000,15000000,45000,1650000.0,1980000.0,0,0.0
6,mobidriven,mp_mobidriven_игроник_национальная лотерия_27.08 (1),мп,Национальная Лотерия,01.11.24-30.04.25,Mobidriven,,in app,Ж 35-45 BC. Семейные с детьми. Работают в офисах или на гос. службе. Интересуются скидками и акциями.,РФ,Ж 35-45 лет,Fullscreen,CPM,319.0,3,12000000,36000000,540000,11484000.0,13780800.0,0,0.0
7,mobidriven,mp_mobidriven_игроник_национальная лотерия_27.08 (1),мп,Национальная Лотерия,01.11.24-30.04.25,Mobidriven,,in web,Ж 35-45 BC. Семейные с детьми. Работают в офисах или на гос. службе. Интересуются скидками и акциями.,РФ,Ж 35-45 лет,"Banners (300*250, 320*100, 320*50)",CPM,121.0,3,10000000,30000000,90000,3630000.0,4356000.0,0,0.0


In [17]:
test.iloc[1]

10
supplier                                                                                                                                                    mobidriven
report_name                                                                                                     mp_mobidriven_игроник_ингосстрах_инго экосистема_05.09
sheet_name                                                                                                                                                          мп
brand                                                                                                                                                       Ингосстрах
period                                                                                                                                                     01.10-31.10
source                                                                                                                                                      Mobidr

In [ ]:
test[['unit quantity', 'reach', 'impressions', 'clicks', 'budget_without_nds']].sum()

In [ ]:
# создаем подключение к Гуглу
client = create_connection(service_key)

In [ ]:
# прописываем путь и подключаемсяк нужному листу Гугл докса
sh = client.open_by_url(google_sheet_link)
sh.share(gmail, perm_type='user', role='writer')
google_sheet = sh.worksheet(sheet_name)

In [ ]:
# выгружем наш датаФрейм на лист
google_sheet.clear()
export_dataframe_to_google_sheet(google_sheet, final_report)

In [ ]:
main_dict['beeline_plan_игроник_ингосстрах_кис_лето_banners_020724']

In [ ]:
main_dict['plan_игроник_рсхб_beeline_свой вклад_banners_160724 (1)'].iloc[0]

In [ ]:
main_dict['plan_ингосстрах_ beeline_хореографы_banners_240924 (3)']

In [ ]:
main_dict['beeline_plan_igronik_бриф по тендерам_101024']